In [191]:
import gzip
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [192]:
yjmob1 = 'yjmob100k-dataset1.csv.gz' # dataset under normal scenes
yjmob_df = pd.read_csv(yjmob1, compression='gzip').sort_values(by=['uid', 'd', 't'], ignore_index=True)

In [214]:
USED_DATA_SIZE = 200 # data used ~ num of uid

In [215]:
# Retrieve all ids
uids = yjmob_df['uid'].unique()

# Select id
rand_indicies = [random.randint(0, len(uids)) for _ in range(USED_DATA_SIZE)] 
selected_uids = [uid for uid in uids[rand_indicies]]

# load data into pd df
df = yjmob_df[yjmob_df['uid'].isin(selected_uids)]

# Time
# df['combined_t'] = df['d']*47+df['t']

# Gird-Location
# got killed
# def spatial_token(x, y):
#     return (x-1)+(y-1)*200
# df['combined_xy'] = df.apply(lambda row: spatial_token(row['x'], row['y']), axis=1)

# Sort values and retrieve only the values of the useful data
df = df.sort_values(by=['uid'])[['uid', 't', 'x']]

```
data_neural = {
    'user1': {
        'sessions': [(loc1, tim1), (loc2, tim2), ...],
        'train': [0, 1, 2],
        'test': [3, 4]
    },
    'user2': {
        'sessions': [(loc1, tim1), (loc2, tim2), ...],
        'train': [0, 1],
        'test': [2]
    },
    ...
}
```

In [216]:
# Split indices into train and test
# 7-3 Train-Test split
def split_indices(n, test_ratio=0.3):
    indices = list(range(n))
    test_size = int(n * test_ratio)
    train_indices = indices[:-test_size] if test_size else indices
    test_indices = indices[-test_size:] if test_size else []
    return train_indices, test_indices

In [217]:
STEP_SIZE = 600

In [218]:
data_neural = {}
WINDOW_SIZE = 10

index = 0

for uid, group in df.groupby('uid'):
    group = group.head(STEP_SIZE)
    sessions = {}
    for i in range(0, len(group), WINDOW_SIZE):
        # Using min to avoid index out of range for the last window
        sessions[i // WINDOW_SIZE] = list(zip( group['x'][i:i + WINDOW_SIZE], 
                                             group['t'][i:i + WINDOW_SIZE]))
    
    # Define the split indices
    num_sessions = len(sessions)
    train_cutoff = int(num_sessions * 0.7)
    
    train_keys = list(sessions.keys())[:train_cutoff]
    test_keys = list(sessions.keys())[train_cutoff:]

    data_neural[index] = {
        'sessions': sessions,
        'train': train_keys,
        'test': test_keys
    }
    
    index += 1

In [219]:
print(len(data_neural))
data_neural

200


{0: {'sessions': {0: [(97, 22),
    (97, 22),
    (103, 23),
    (102, 24),
    (103, 25),
    (103, 26),
    (102, 27),
    (102, 28),
    (102, 29),
    (108, 30)],
   1: [(103, 12),
    (102, 13),
    (102, 15),
    (102, 16),
    (102, 18),
    (102, 19),
    (102, 20),
    (97, 21),
    (102, 15),
    (102, 14)],
   2: [(103, 13),
    (107, 12),
    (108, 31),
    (103, 30),
    (102, 21),
    (102, 29),
    (102, 27),
    (103, 26),
    (102, 25),
    (103, 24)],
   3: [(102, 23),
    (103, 22),
    (102, 28),
    (103, 16),
    (103, 20),
    (105, 18),
    (132, 30),
    (132, 31),
    (133, 32),
    (132, 35)],
   4: [(133, 36),
    (121, 37),
    (110, 38),
    (94, 39),
    (95, 42),
    (94, 43),
    (108, 44),
    (99, 45),
    (96, 46),
    (113, 0)],
   5: [(108, 22),
    (98, 29),
    (107, 30),
    (103, 17),
    (102, 16),
    (103, 15),
    (102, 14),
    (103, 13),
    (102, 12),
    (103, 19)],
   6: [(108, 41),
    (115, 36),
    (158, 35),
    (151, 34),
    (151

In [220]:
output = {}

In [221]:
uid_list = [int(i) for i in range(len(data_neural))]

In [222]:
output['data_neural'] = data_neural
output['uid_list'] = uid_list

In [223]:
# Save the dictionary to a pickle file
import pickle

with open('my_data_neural.pk', 'wb') as f:
    pickle.dump(output, f)